<a href="https://colab.research.google.com/github/maiormarso/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/LS__DS__212__assignment_regression_ll.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [X] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [X] Engineer at least two new features. (See below for explanation & ideas.)
- [X] Fit a linear regression model with at least two features.
- [X] Get the model's coefficients and intercept.
- [X] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [X] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [X] As always, commit your notebook to your fork of the GitHub repo.



#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [0]:
df.shape

(48817, 34)

In [0]:
df['bednbaths'] = df['bedrooms'] + df['bathrooms']

In [0]:
df['catsndogs_allowed'] = df['cats_allowed'] + df['dogs_allowed']
df.shape

(48817, 36)

In [0]:
df['year'] =  pd.DatetimeIndex(df['created']).year

In [0]:
df['month'] = pd.DatetimeIndex(df['created']).month

In [0]:
df.head(1)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,bednbaths,catsndogs_allowed,year,month
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.5,0,2016,6


In [0]:
array = ['4','5']
train= df.loc[(df['year'] >= 2016) & df['month'].isin(array)]
train.head(500)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,bednbaths,catsndogs_allowed,year,month
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0,2016,4
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0,2016,4
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.0,0,2016,4
5,2.0,4,2016-04-19 04:24:47,,West 18th Street,40.7429,-74.0028,7995,350 West 18th Street,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.0,0,2016,4
6,1.0,2,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,40.8012,-73.9660,3600,210 West 107th Street,low,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,2,2016,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,1.0,3,2016-04-05 06:23:14,This is a beautiful 3 bedroom!SunnyHardwood fl...,Dean St.,40.6754,-73.9182,2000,2097 Dean St.,medium,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0,0,2016,4
547,2.0,2,2016-04-07 06:24:51,NO FEE + 1 month free*Fantastic location in th...,West 48th Street,40.7609,-73.9860,5350,235 West 48th Street,low,1,0,1,0,1,1,1,0,1,1,0,1,0,0,1,0,1,0,0,0,0,1,0,0,4.0,0,2016,4
548,1.0,1,2016-04-21 05:02:03,Newly constructed luxury high rise in the west...,W 42 St.,40.7611,-73.9989,3800,605 W 42 St.,low,1,0,1,0,1,1,1,0,1,0,1,1,1,0,0,0,1,1,0,0,0,0,0,0,2.0,0,2016,4
549,1.0,1,2016-04-17 03:32:17,LOCATION: 137th b/w Adam Clayton Powell and Ma...,W 137th,40.8163,-73.9416,1850,137 W 137th,low,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,2,2016,4


In [0]:
array = ['6']
test=df.loc[(df['year'] >= 2016) & df['month'].isin(array)]
test.head(500)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,bednbaths,catsndogs_allowed,year,month
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.5,0,2016,6
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,2,2016,6
11,1.0,1,2016-06-03 03:21:22,Check out this one bedroom apartment in a grea...,W. 173rd Street,40.8448,-73.9396,1675,644 W. 173rd Street,low,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0,2016,6
14,1.0,1,2016-06-01 03:11:01,Spacious 1-Bedroom to fit King-sized bed comfo...,East 56th St..,40.7584,-73.9648,3050,315 East 56th St..,low,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0,2016,6
24,2.0,4,2016-06-07 04:39:56,SPRAWLING 2 BEDROOM FOUND! ENJOY THE LUXURY OF...,W 18 St.,40.7391,-73.9936,7400,30 W 18 St.,medium,1,1,1,1,1,1,0,0,1,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,6.0,2,2016,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5026,2.5,2,2016-06-22 02:21:51,Tribecas Finest and Most Luxurious Full Servic...,Barclay St.,40.7121,-74.0086,9500,10 Barclay St.,low,1,0,1,0,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,4.5,0,2016,6
5033,1.0,1,2016-06-21 01:42:05,This apartment is located in a walk up buildin...,Convent Avenue,40.8169,-73.9529,1825,102 Convent Avenue,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0,2016,6
5038,1.0,2,2016-06-02 03:26:48,"NICE 2 BR apartment on great location, new kit...",First Avenue,40.7659,-73.9574,2600,1685 First Avenue,low,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,1,2016,6
5044,1.0,0,2016-06-02 03:10:55,***Large Studio in the Upper East Side*** -P/T...,East 86th Street,40.7771,-73.9505,2450,336 East 86th Street,medium,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,2016,6


In [0]:
avg_rent = train['price'].mean()
print(f'Average price to rent an appartment in NY City is ${avg_rent:,.0f},')

Average price to rent an appartment in NY City is $3,576,


In [0]:
# 1. Import the appropriate estimater class for Scikit-learn
from sklearn.linear_model import LinearRegression

In [0]:
# 2. Instantiate this class
model = LinearRegression() 

In [0]:
import numpy as np
>>> from sklearn.model_selection import train_test_split

In [0]:
#3. Arrange X features matrix * y target vector'
features =['bathrooms','bedrooms']
target = 'price'
X_train = df[features]
y_train = df[target]
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25)



In [0]:
X_test.head()

,bathrooms,bedrooms
9392,1.5,2
40637,1.0,2
44689,1.0,2
33913,1.5,2
34257,2.0,2


In [0]:
# Get mean baseline
print('Mean Baseline (using 0 features')
guess = y_train.mean()
print(guess)

Mean Baseline (using 0 features
3579.379684256528


In [0]:
# Train Error
from sklearn.metrics import mean_absolute_error
y_pred = [guess] * len(y_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train Error (NY City Apartment Rentals): ${mae:.2f} dollars')

Train Error (NY City Apartment Rentals): $1202.98 dollars


In [0]:
# Test Error
y_pred = [guess] * len(y_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Error (NY City Apartment Rentals): {mae:.2f} dollars')

Test Error (NY City Apartment Rentals): 1196.97 dollars


In [0]:
#4 Fit the model
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

#RMSE MSE R^2

In [0]:
from sklearn.metrics import mean_squared_error, r2_score
y_test_prediction = model.predict(X_test)
MSE = mean_squared_error(y_test, y_test_prediction)
RMSE = np.sqrt(mean_squared_error(y_test, y_test_prediction))
MSE,RMSE

R2 = r2_score(y_test, y_test_prediction)
R2


0.5138137828212737

In [0]:
import category_encoders as ce

encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

X_train_encoded.head()

,bathrooms,bedrooms
29186,1.0,0
45391,1.0,0
20947,1.0,0
40172,1.0,0
7950,1.0,0


In [0]:
# 5. Apply the model to new
bedrooms = 1
bathrooms = 1.5
X_test = [[bedrooms,bathrooms]]
y_pred = model.predict(X_test)
                      
y_pred  

array([3139.22384246])

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
def lr(train, test, features, target):
    X_train = train[features]
    y_train = train[target]
    X_test = test[features]
    y_test = test[target]

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    print(f'Linear Regression with {len(features)} features')
    print('Intercept', model.intercept_)
    coefficients = pd.Series(model.coef_, features)
    print(coefficients.to_string())
    
    print('Train Root Mean Squared Error:', np.sqrt(mean_squared_error(y_train, y_pred_train)))
    print('Test Root Mean Square Error:', np.sqrt(mean_squared_error(y_test, y_pred_test)))
    print('Train Mean Absolute Error:', mean_absolute_error(y_train, y_pred_train))
    print('Test Mean Absolute Error:', mean_absolute_error(y_test, y_pred_test))
    print('Train R^2 Score:', r2_score(y_train, y_pred_train))
    print('Test R^2 Score:', r2_score(y_test, y_pred_test))
lr(train, test, features, target)

Linear Regression with 2 features
Intercept 485.71869002322865
bathrooms    2072.610116
bedrooms      389.324896
Train Root Mean Squared Error: 1232.0225917223484
Test Root Mean Square Error: 1219.719357233823
Train Mean Absolute Error: 818.5310213271714
Test Mean Absolute Error: 825.8987822403527
Train R^2 Score: 0.5111543084316607
Test R^2 Score: 0.5213303957090345


In [0]:
model.coef_

array([2099.90713798,  378.80806756])

In [0]:
model.intercept_

471.10460313868725

In [0]:
# Equations for a line
m = model.coef_[0]
b = model.intercept_

print('y =mx + b')
print(f'y = {m: 0f}*X + {b: 0f}')
print(f'y - {m: 0f}*X + {b: 0f}')
print(f'price = {m:0f}*bedrooms + {b:.0f}')   

y =mx + b
y =  2099.907138*X +  471.104603
y -  2099.907138*X +  471.104603
price = 2099.907138*bedrooms + 471


In [0]:
def predict(bedrooms,bathrooms): 
  y_pred= model.predict([[bedrooms,bathrooms]])
  estimate= y_pred[0]
  coefficient = model.coef_[1]
  result = f'${estimate:,.0f} estimated price for {bedrooms:,.0f} bedroom apparment.'
  explanation = f'in this linear regression each additional bedroom adds ${coefficient:,.0f}.'
  return result + explanation

In [0]:
predict(1, 2) 

'$3,329 estimated price for 1 bedroom apparment.in this linear regression each additional bedroom adds $379.'

I read the two articles for stretch goal and I added some items along the way for the stretch goal.